In [10]:
# Import libraries

import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [11]:
# Import World Bank Tourism Arrival CSV file

df_arrivals_original = pd.read_csv('../data/raw/API_ST.INT.ARVL_DS2_en_csv_v2_1740745.csv', header=2)
df_arrivals = df_arrivals_original.copy()

df_arrivals.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,619000.0,641000.0,650000.0,647000.0,683000.0,721000.0,691000.0,643000.0,642000.0,728000.0,733000.0,694000.0,772000.0,827000.0,813000.0,824000.0,869000.0,904000.0,979000.0,1072000.0,1225000.0,1102000.0,1070500.0,1082000.0,NaN,NaN,NaN
1,Afghanistan,AFG,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9000.0,21000.0,45000.0,52000.0,45000.0,51000.0,67000.0,91000.0,107000.0,194000.0,210000.0,121000.0,195000.0,294000.0,366000.0,425000.0,481000.0,528000.0,650000.0,595000.0,592000.0,397000.0,261000.0,218000.0,NaN,NaN,NaN
3,Albania,ALB,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1062000.0,1247000.0,1711000.0,2191000.0,2469000.0,3156000.0,2857000.0,3341000.0,3784000.0,4070000.0,4643000.0,5340000.0,NaN,NaN,NaN
4,Andorra,AND,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2347000.0,2949000.0,3516000.0,3387000.0,3138000.0,2791000.0,2418000.0,2227000.0,2189000.0,2059000.0,1830000.0,1808000.0,2242000.0,2238000.0,2328000.0,2363000.0,2663000.0,2819000.0,3003000.0,3042000.0,NaN,NaN,NaN


In [12]:
# Filter Tourism Arrival to keep needed columns (years)

cols_keep = ['Country Name', 'Country Code']
years_keep = [str(x) for x in [*range(2012,2019)]] # unpacks range into a list of strings
cols_keep = cols_keep + years_keep
df_arrivals = df_arrivals[cols_keep]

# Drop countries (rows) with any missing values in relevant years

df_arrivals = df_arrivals.dropna()

# Convert from wide to long format

df_arrivals = pd.melt(df_arrivals, id_vars =['Country Name', 'Country Code'], value_vars = years_keep) 

#print(df_arrivals.dtypes)
df_arrivals.head()

,Country Name,Country Code,variable,value
0,Aruba,ABW,2012,9.040000e+05
1,Angola,AGO,2012,5.280000e+05
2,Albania,ALB,2012,3.156000e+06
3,Andorra,AND,2012,2.238000e+06
4,Arab World,ARB,2012,8.484459e+07


In [13]:
# Import World Bank Population XML file

# XML parsing code adapted from:
# https://stackabuse.com/reading-and-writing-xml-files-in-python-with-pandas/

xml_data = open('../data/raw/API_SP.POP.TOTL_DS2_en_xml_v2_1741834.xml', 'r').read()  # Read file

root = ET.XML(xml_data)  # Parse XML
data = []
xml_cols = []
for i, child in enumerate(root[0]):
    data.append([subchild.text for subchild in child])

df_population_original = pd.DataFrame(data)  # Write in DF
df_population = df_population_original.copy()

df_population.columns = ['Country Name', 'Indicator Name', 'Year', 'Population'] 
df_population = df_population[['Country Name', 'Year', 'Population']]

df_population = df_population.pivot(index='Country Name', columns='Year', values='Population').reset_index()

df_population.head()

## NOTE: These results are alphabetized by country name, not by code like the
## tourism arrival CSV file below

Year,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Afghanistan,8996973,9169410,9351441,9543205,9744781,9956320,10174836,10399926,10637063,10893776,11173642,11475445,11791215,12108963,12412950,12689160,12943093,13171306,13341198,13411056,13356511,13171673,12882528,12537730,12204292,11938208,11736179,11604534,11618005,11868877,12412308,13299017,14485546,15816603,17075727,18110657,18853437,19357126,19737765,20170844,20779953,21606988,22600770,23680871,24726684,25654277,26433049,27100536,27722276,28394813,29185507,30117413,31161376,32269589,33370794,34413603,35383128,36296400,37172386,38041754,None
1,Albania,1608800,1659800,1711319,1762621,1814135,1864791,1914573,1965598,2022272,2081695,2135479,2187853,2243126,2296752,2350124,2404831,2458526,2513546,2566266,2617832,2671997,2726056,2784278,2843960,2904429,2964762,3022635,3083605,3142336,3227943,3286542,3266790,3247039,3227287,3207536,3187784,3168033,3148281,3128530,3108778,3089027,3060173,3051010,3039616,3026939,3011487,2992547,2970017,2947314,2927519,2913021,2905195,2900401,2895092,2889104,2880703,2876101,2873457,2866376,2854191,None
2,Algeria,11057863,11336339,11619828,11912803,12221675,12550885,12902627,13275026,13663583,14061722,14464985,14872250,15285990,15709825,16149025,16607707,17085801,17582904,18102266,18647815,19221665,19824301,20452902,21101875,21763575,22431502,23102389,23774284,24443467,25106190,25758869,26400479,27028326,27635515,28213774,28757785,29266405,29742979,30192754,30623406,31042235,31451514,31855109,32264157,32692163,33149724,33641002,34166972,34730608,35333881,35977455,36661444,37383887,38140132,38923687,39728025,40551404,41389198,42228429,43053054,None
3,American Samoa,20123,20602,21253,22034,22854,23672,24462,25248,25989,26703,27363,27984,28567,29100,29596,30052,30456,30838,31269,31845,32646,33701,34968,36412,37946,39519,41119,42740,44343,45894,47347,48685,49896,51020,52095,53161,54211,55221,56171,57053,57821,58494,59080,59504,59681,59562,59107,58365,57492,56683,56079,55759,55667,55713,55791,55812,55741,55620,55465,55312,None
4,Andorra,13411,14375,15370,16412,17469,18549,19647,20758,21890,23058,24276,25559,26892,28232,29520,30705,31777,32771,33737,34818,36067,37500,39114,40867,42706,44600,46517,48455,50434,52448,54509,56671,58888,60971,62677,63850,64360,64327,64142,64370,65390,67341,70049,73182,76244,78867,80993,82684,83862,84463,84449,83747,82427,80774,79213,78011,77297,77001,77006,77142,None


In [14]:
# Filter Population to keep needed columns (years)

cols_keep = ['Country Name']
cols_keep = cols_keep + years_keep
df_population = df_population[cols_keep]

# Drop countries (rows) with any missing values in relevant years

df_population = df_population.dropna()

# Convert from wide to long format

df_population = pd.melt(df_population, id_vars =['Country Name'], value_vars = years_keep) 

# df_population.dtypes
df_population.head()

,Country Name,Year,value
0,Afghanistan,2012,31161376
1,Albania,2012,2900401
2,Algeria,2012,37383887
3,American Samoa,2012,55667
4,Andorra,2012,82427


In [15]:
# Join Tourism Arrival and Population dfs

tourism_pop_df = df_arrivals.merge(df_population, left_on=['Country Name', 'variable'], 
                                   right_on = ['Country Name', 'Year'], how='left')

# Clean up columns and column names

tourism_pop_df = tourism_pop_df.rename(columns={"Country Name": "country_txt_wb", 
                                "Country Code": "country_code", 
                                "variable": "year_wb", "value_x": "arrivals", 
                                "value_y": "population"})
tourism_pop_df = tourism_pop_df[["country_txt_wb", "country_code", "year_wb", 
                                 "arrivals", "population"]]

tourism_pop_df.head()

,country_txt_wb,country_code,year_wb,arrivals,population
0,Aruba,ABW,2012,9.040000e+05,102560
1,Angola,AGO,2012,5.280000e+05,25107931
2,Albania,ALB,2012,3.156000e+06,2900401
3,Andorra,AND,2012,2.238000e+06,82427
4,Arab World,ARB,2012,8.484459e+07,371443547


In [16]:
# Import mapping CSV file between GTD country IDs and WB country codes (CSV file)

df_country_mapping_original = pd.read_csv('../data/raw/gtd_to_wb_country_mapping.csv', header=0)
df_country_mapping = df_country_mapping_original.copy()
df_country_mapping = df_country_mapping.drop(df_country_mapping.index[0])

df_country_mapping.head()

,country_id,gtd_name,wb_country_code,wb_name
1,4.0,Afghanistan,AFG,Afghanistan
2,5.0,Albania,ALB,Albania
3,6.0,Algeria,DZA,Algeria
4,7.0,Andorra,AND,Andorra
5,8.0,Angola,AGO,Angola


In [17]:
# Join GTD country IDs to final tourism + Population dataframe

tourism_pop_df = tourism_pop_df.merge(df_country_mapping, left_on=['country_code'], 
                                   right_on = ['wb_country_code'], how='left')
tourism_pop_df = tourism_pop_df.drop(['gtd_name', 'wb_country_code', 'wb_name'], axis=1)

# Clean up by reordering columns and casting appropriate columns as integers

tourism_pop_df = tourism_pop_df[['year_wb', 'country_id', 'country_txt_wb', 'country_code', 'arrivals', 'population']]

# Drop countries (rows) where there is not a country_id (from GTD) for the WB country code

tourism_pop_df = tourism_pop_df.dropna()
tourism_pop_df = tourism_pop_df.reset_index(drop=True)

# Cast relevant columns to integers

tourism_pop_df['year_wb'] = tourism_pop_df['year_wb'].astype(float).astype(int)
tourism_pop_df['country_id'] = tourism_pop_df['country_id'].astype(float).astype(int) 
tourism_pop_df['arrivals'] = tourism_pop_df['arrivals'].astype(float).astype(int)
tourism_pop_df['population'] = tourism_pop_df['population'].astype(float).astype(int)

#print(tourism_pop_df.dtypes)
tourism_pop_df.head()

,year_wb,country_id,country_txt_wb,country_code,arrivals,population
0,2012,8,Angola,AGO,528000,25107931
1,2012,5,Albania,ALB,3156000,2900401
2,2012,7,Andorra,AND,2238000,82427
3,2012,11,Argentina,ARG,6497000,41733271
4,2012,12,Armenia,ARM,963000,2884229


In [18]:
# Write final output of WB Tourism and Poluation data (with a matched GTD county_id) to a CSV file

tourism_pop_df.to_csv('../data/derived/tourism_pop.csv')